In [6]:
# Load libraries
import lwt_functions
import pandas as pd
from dateutil import parser
import isodate
import datetime
import json
import urllib
import re
import boto3
import os
import time
from pydub import AudioSegment
import glob



# import nltk
# from nltk.tokenize import word_tokenize, sent_tokenize
# from nltk.corpus import stopwords
# import matplotlib.pyplot as plt
# from PIL import Image
# import numpy as np
# from wordcloud import WordCloud, STOPWORDS

# Set pandas display settings
pd.options.display.max_colwidth = None
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

#### Scrape wikipedia page for LWT episodes

In [ ]:
# Scrape the tables from the LWT Wikipedia page 
wiki_url = 'https://en.wikipedia.org/wiki/List_of_Last_Week_Tonight_with_John_Oliver_episodes'
all_tables = lwt_functions.scrape_wikipedia_tables(wiki_url)

In [ ]:
# Filter to those tables that contain episode information
season_tables = [table for table in all_tables if 'wikiepisodetable' in table['class']]

# Iterate through seasons and compile episode information
episode_number, main_segment_title, air_date, viewers = [], [], [], []
for season in season_tables: 
    for row in season.findAll('tr'):
        cells = row.findAll('td')
        if len(cells) == 4:
            episode_number.append(cells[0].find(text=True))
            main_segment_title.append(cells[1].findAll(text=True))
            air_date.append(cells[2].find(text=True))
            viewers.append(cells[3].find(text=True))

# Correcting issues with compiled information
air_date = [unicodedata.normalize('NFKC', date) for date in air_date]
main_segment_title = [''.join(title).strip() for title in main_segment_title]
viewers = [float(v)*1000000 for v in viewers if v != 'TBD']

# Create list to track the episode's corresponding season number
season = []
season_number = 0
for episode in episode_number:
    if episode == '1':
        season_number += 1
    season.append(season_number)

In [ ]:
# Convert individual field lists into DataFrame
lwt_episodes_wiki = pd.DataFrame([season, episode_number, main_segment_title, air_date, viewers])
lwt_episodes_wiki = lwt_episodes_wiki.transpose()
lwt_episodes_wiki.columns = ['season','episode','main_segment_title','air_date','viewers']
lwt_episodes_wiki['episode_overall'] = lwt_episodes_wiki.index + 1
lwt_episodes_wiki['air_date'] = pd.to_datetime(lwt_episodes_wiki['air_date'])
lwt_episodes_wiki[lwt_episodes_wiki.main_segment_title != ''].shape

In [ ]:
# Output results to CSV
lwt_episodes_wiki.to_csv('data/lwt_episodes_wiki.csv', header=True, index=False)

#### Get info for individual videos from LWT's YouTube channel

In [ ]:
# Get API key generated on GCP 
with open('reference/youtube_api_key.txt') as f:
    api_key = f.read()

In [ ]:
# Iterate through LWT channel's videos
lwt_youtube_channel = 'UC3XTzVzaHQEd30rQbuvCtTQ'
videos = lwt_functions.fetch_all_youtube_videos(lwt_youtube_channel, api_key)

lst = []
for video in videos['items']:
    video_stats = lwt_functions.get_statistics(video['id']['videoId'], api_key)
    results_json = {
        'channelTitle':video['snippet']['channelTitle'],
        'title':video['snippet']['title'],
        'publishedAt':video['snippet']['publishedAt'],
        'videoId':video['id']['videoId'],
        'duration':video_stats['items'][0]['contentDetails']['duration'],
        'viewCount':video_stats['items'][0]['statistics']['viewCount'],
        'commentCount':video_stats['items'][0]['statistics']['commentCount'],
        'likeCount':video_stats['items'][0]['statistics']['likeCount'],
        'dislikeCount':video_stats['items'][0]['statistics']['dislikeCount']
    }

    lst.append(results_json)
    
# Convert list to DataFrame and output to avoid re-running
lwt_episodes_yt = pd.read_json(json.dumps(lst))
print(lwt_episodes_yt.shape)

In [ ]:
lwt_episodes_yt.sample(1)

In [ ]:
# Confirm the first episode of LWT (Season 1, Epsidoe 1) is missing from YouTube
print(main_segments_yt.publishedAt.dt.date.min())
print(lwt_episodes_wiki.air_date.dt.date.min())

In [ ]:
# Export YouTube results to CSV to prevent hitting API quota
lwt_episodes_yt.to_csv('data/lwt_episodes_yt.csv', header=True, index=False)

#### Inspect the DOW episodes originally aired on HBO vs YouTube

In [ ]:
# Test whether all LWT episodes aired on Sunday
lwt_episodes_wiki = pd.read_csv('data/lwt_episodes_wiki.csv', header=0, parse_dates=['air_date'])
print(f'Total LWT Episodes (before today): {lwt_episodes_wiki[lwt_episodes_wiki.air_date < datetime.datetime.now() - datetime.timedelta(days=1)].shape[0]}') # ran on Sunday so exlcuded today's episode
print(f"Episodes not on Sunday: {lwt_episodes_wiki[lwt_episodes_wiki['air_date'].dt.day_name() != 'Sunday'].sum().sum():.0f}")

In [ ]:
# Inspect when LWT episodes appeared on YT
lwt_episodes_yt = pd.read_csv('data/lwt_episodes_yt.csv', header=0, parse_dates=['publishedAt'])
lwt_episodes_yt['published_date'] = lwt_episodes_yt['publishedAt'].dt.date
lwt_episodes_yt['published_dow'] = lwt_episodes_yt['publishedAt'].dt.day_name()
pd.DataFrame(lwt_episodes_yt.published_dow.value_counts(normalize=False, sort=True))

#### Filter YouTube videos to find main segments

In [ ]:
# Engineer additional fields
lwt_episodes_yt['main_segment_title'] = lwt_episodes_yt['title'].str.split(':').str[0].str.replace('&#39;',"'").str.replace('&quot;','"')
lwt_episodes_yt['duration_in_seconds'] = lwt_episodes_yt['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds())
lwt_episodes_yt['duration_time'] = lwt_episodes_yt['duration'].apply(lambda x: str(datetime.timedelta(seconds = isodate.parse_duration(x).total_seconds())))

In [ ]:
# Filter out videos we know aren't main segments based on their title
filter_phrases = ['web exclusive','how is this still a thing','official trailer','extended interview','dancing zebra footage','mercadeo ']
main_segments_yt = lwt_episodes_yt.copy()
for phrase in filter_phrases:
    main_segments_yt = main_segments_yt[~main_segments_yt['title'].str.lower().str.contains(phrase)]

# Filter out videos less than 5 minutes in duration
# Assuming main segments are typically longer
main_segments_yt = main_segments_yt[main_segments_yt.duration_in_seconds >= 60*5]

# Filter videos that didn't get published to YouTube on Monday
main_segments_yt = main_segments_yt[main_segments_yt.published_dow == 'Monday']
main_segments_yt.published_dow.value_counts()

In [ ]:
# Find which dates have 2+ videos
# There are 9 dates with 2+ videos published
yt_published_date_count = pd.DataFrame(main_segments_yt.groupby(['published_date'])['videoId'].count()).reset_index()
yt_published_date_count_two_plus = yt_published_date_count[yt_published_date_count.videoId >= 2]
print(yt_published_date_count_two_plus.shape)

In [ ]:
# Find the videos on those dates with the longest duration 
duplicate_dates = main_segments_yt[main_segments_yt.published_date.isin(yt_published_date_count_two_plus.published_date)].sort_values(by='published_date')
duplicate_dates['duration_rank'] = duplicate_dates.groupby('published_date')['duration_in_seconds'].rank("dense", ascending=False)
duplicate_dates = duplicate_dates[duplicate_dates.duration_rank > 1]
print(duplicate_dates.shape)

# Remove shorter videos  from main segments dataframe
main_segments_yt = main_segments_yt[~main_segments_yt.videoId.isin(duplicate_dates.videoId)]
main_segments_yt.shape

#### Merge the Youtube and Wiki datasets

In [ ]:
# Prepare dataframes to merge
lwt_episodes_wiki = lwt_episodes_wiki.reset_index()
lwt_episodes_wiki = lwt_episodes_wiki[lwt_episodes_wiki.air_date < datetime.datetime.now() - datetime.timedelta(days=1)] # running on Sunday so need to exclude today's episode
lwt_episodes_wiki['wiki_join_field'] = lwt_episodes_wiki.air_date + pd.DateOffset(1)
lwt_episodes_wiki.set_index('wiki_join_field', inplace=True)
assert lwt_episodes_wiki.index.duplicated().sum() == 0

main_segments_yt = main_segments_yt.reset_index()
main_segments_yt.set_index('published_date', inplace=True)
main_segments_yt.index = pd.to_datetime(main_segments_yt.index)
assert main_segments_yt.index.duplicated().sum() == 0

In [ ]:
# Join the wiki to YouTube datasets based on air/published date
lwt_episodes = lwt_episodes_wiki.join(main_segments_yt, how='left', on=lwt_episodes_wiki.index, lsuffix='_wiki', rsuffix='_yt')
print(lwt_episodes.shape)
print(f'Missing # of YouTube videos: {lwt_episodes.videoId.isnull().sum()}') # expected result is 1 since missing first episode

In [ ]:
# Push merged data to CSV to prevent re-running cells above
lwt_episodes.to_csv('data/lwt_episodes.csv', header=True, index=True)

#### Convert YouTube videos to audio files

In [3]:
# Prep lists of urls and corresponding filesnames
lwt_episodes = pd.read_csv('data/lwt_episodes.csv', header=0, index_col=0, parse_dates=['air_date','publishedAt'])
urls = [f'https://www.youtube.com/watch?v={v}' for v in lwt_episodes.videoId.values if not pd.isna(v)]
filenames = [re.sub('[^0-9a-zA-Z ]+', '', t.lower()) for t in lwt_episodes.main_segment_title_wiki.values][1:]
print(len(urls),len(filenames))

197 197


In [5]:
filenames[68], filenames[165]

('scientific research and science journalism', 'boris johnson')

In [ ]:
# Download YouTube videos as mp3s
# Failed: 68, 165

try:
    for u, f in zip(urls[68:69], filenames[68:69]):
        # pass # adding PASS to prevent accidental re-run
        lwt_functions.download_youtube_video_mp3(u, f)
except:
    print(f'FAILED: {u} {f}')

#### Transcribe videos using AWS Transcribe

Since I'm using my own personal account, I wanted to get a sense for how much this step in the project would cost). Given the pricing structure below (from https://aws.amazon.com/transcribe/pricing/), we have 197 episodes, each other no longer than about 20 minutes. Using the  10 and 30 minutes prices to get an estimated range, we're looking at a ballpark price between 47 and 142 bucks.

<img src="images/aws_transcribe_pricing.PNG" width="480">

In [ ]:
# Get list of audio files
audio_files = os.listdir('audio')
print(len(audio_files))

In [ ]:
# Upload files to S3 bucket
s3 = boto3.client('s3')
for file in audio_files:
    with open(f'audio/{file}', 'rb') as f:
        s3.upload_fileobj(f, 'last-week-tonight-audio-for-transcription', file)

In [ ]:
# Verify all 197 videos uploaded (minus 2 that failed mp3 download)
s3 = boto3.client('s3')
keys = []
for key in s3.list_objects(Bucket='last-week-tonight-audio-for-transcription')['Contents']:
    keys.append(key['Key'])
print(len(keys))

In [ ]:
# Turns out that mp3 is not a supported type so converting files to .wav
from pydub.utils import which
AudioSegment.converter = which("ffmpeg")

for file in audio_files:
    sound = AudioSegment.from_file(f"audio/{file}")
    sound.export(f"audio/{file.split('.mp3')[0]}.wav", format="wav")

In [ ]:
# Move old mp3 files to new folder JIC
mp3_files = glob.glob('audio/*.mp3', )
mp3_files = [f.split('\\')[1] for f in mp3_files]

if not os.path.exists('audio_mp3'):
    os.mkdir('audio_mp3')
    
for file in mp3_files:
    os.rename(f"audio/{file}", f"audio_mp3/{file}")

In [ ]:
# Delete files from S3 bucket
client = boto3.client('s3')
for k in keys:
    client.delete_object(Bucket='last-week-tonight-audio-for-transcription', Key=k)

In [ ]:
# Pass audio files in S3 to Transcribe
transcribe = boto3.client('transcribe', region_name='us-east-1')

for i, audio in enumerate(audio_files[44:]):
    job_name = f'Transcribe_{i}_{audio.replace(" ","_").replace("(","_").replace(")","_").replace("&","and")}'
    job_uri = f'https://last-week-tonight-audio-for-transcription.s3.amazonaws.com/{audio.replace(" ","+")}'
    transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': job_uri},
        MediaFormat='wav',
        LanguageCode='en-US')

In [7]:
# Get list of transcription jobs
# Other statuses-> 'QUEUED'|'IN_PROGRESS'|'FAILED'|
client = boto3.client('transcribe', region_name='us-east-1')
response = client.list_transcription_jobs(
    Status='COMPLETED',
    JobNameContains='Transcribe_',
    #NextToken='string',
    MaxResults=100)

response2 = client.list_transcription_jobs(
    Status='COMPLETED',
    JobNameContains='Transcribe_',
    NextToken=response['NextToken'],
    MaxResults=100)


In [8]:
# Compile job names
job_names = []
for job in response['TranscriptionJobSummaries']:
    job_names.append(job['TranscriptionJobName'])
for job in response2['TranscriptionJobSummaries']:
    job_names.append(job['TranscriptionJobName'])
print(len(job_names))

195


In [ ]:
for job in job_names:
    status = client.get_transcription_job(TranscriptionJobName=job)
    if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
        response = urllib.request.urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
        data = json.loads(response.read())
        with open(f"transcription/{job[:-4]}.json", 'w') as fp:
            json.dump(data, fp)

In [9]:
# Delete transcription jobs
transcribe = boto3.client('transcribe', region_name='us-east-1')
for job in job_names:
    client.delete_transcription_job(TranscriptionJobName=job)